<A HREF="https://theaidigest.in/topic-modeling-using-roberta-and-transformers/">Topic modeling using Roberta and transformers</A><BR>
    <A HREF="https://maartengr.github.io/BERTopic/index.html">BERTopic documentation</A>

In [ ]:
!pip3 install bertopic

In [ ]:
!python3 -m spacy download en_core_web_md

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import en_core_web_md
import csv
import regex as re
import os
import spacy
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim import corpora
import pprint

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook() #Notebook visualisation enabled

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

from sentence_transformers import SentenceTransformer

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

/home/muddy/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/muddy/.local/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/muddy/.local/lib/python3.10/site-packages/joblib/backports.py:22: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils  # noqa
/home/muddy/.local/lib/python3.10/site-packages/bertopic/_utils.py:4: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_

In [2]:
# Load up the files
#path = './DataUCSB/'
path = './Data/'
list_of_files = []

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.txt'):
            list_of_files.append(os.path.join(root,file))

filepath = './DataUCSB/address-before-joint-session-the-congress-the-state-the-union-16.csv'
speeches = []
for file in list_of_files:
    with open(file, encoding='utf-8') as f:
        text = f.read()
    speeches.append(text)

In [3]:
listlist = [speech.split('.') for speech in speeches]
sentences = listlist[0]
# Prepare embeddings
# all-mpnet-base-v2 is about 400MB
sentence_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = sentence_model.encode(sentences)

In [4]:
import nltk
from nltk.corpus import stopwords

In [ ]:
#Load SpaCy English Model
nlp = en_core_web_md.load()

#Tags to remove
extags = ['PRON','CCONJ','PUNCT','PART','DET','ADP','NUM','SYM','SPACE']

tokens=[]
#SpaCy tokenization + lemmatization + lowercase
for speech in nlp.pipe(speeches):
    scr_tok = [token.lemma_.lower() for token in speech if token.pos_ not in extags and not token.is_stop and token.is_alpha]
    tokens.append(scr_tok)

<A HREF="https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html#removing-stop-words">Removing stop words</A> a little different than sometimes
<P>
At times, stop words might end up in our topic representations. This is something we typically want to avoid as they contribute little to the interpretation of the topics. However, removing stop words as a preprocessing step is not advised as the transformer-based embedding models that we use need the full context in order to create accurate embeddings.
</P>
<P>
Instead, we can use the CountVectorizer to preprocess our documents after having generated embeddings and clustered our documents. Personally, I have found almost no disadvantages to using the CountVectorizer to remove stopwords and it is something I would strongly advise to try out:
</P>

<A HREF="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html">sklearn CountVectorizer docs</A>

In [ ]:
# This seems to lock up or goes on an endless loop
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [5]:
stop_words = text.ENGLISH_STOP_WORDS.union(['thats', 'just', 'im', 'did', 'thing', 'mr'])
#vectorizer_model = CountVectorizer(stop_words=stop_words, tokenizer=LemmaTokenizer())
# stop_words="english"
vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=(1,3))
#vectorizer_model = CountVectorizer(max_df=0.99)
model = BERTopic(vectorizer_model=vectorizer_model)
topics, probs = model.fit_transform(speeches)

AttributeError: 'str' object has no attribute 'ENGLISH_STOP_WORDS'

In [6]:
#topics = model.fit(sentences)
vectorizer_model = CountVectorizer(stop_words='english')
model = BERTopic(vectorizer_model=vectorizer_model)
topics, probs = model.fit_transform(sentences, embeddings)

In [7]:
model.get_topic_info()

,Topic,Count,Name
0,0,78,0_know_children_time_love
1,1,21,1_catherine_dylan_charlotte_emilie


The -1 topic is an outlier topic and contains documents that are generally unrelated to one another. The documents in that topic are essentially random in nature and therefore do not represent an actual topic. Thus, the -1 topic cannot be interpreted as all other topics. <A HREF="https://github.com/MaartenGr/BERTopic/issues/677">reference</A>

In [8]:
model.get_topic(0)

[('know', 0.07573764051107666),
 ('children', 0.06901112028813906),
 ('time', 0.050255556749243896),
 ('love', 0.046079021325495406),
 ('newtown', 0.04171936001037425),
 ('youve', 0.03714901624350276),
 ('thats', 0.03714901624350276),
 ('community', 0.03233112224233856),
 ('comfort', 0.03233112224233856),
 ('say', 0.03233112224233856)]

In [9]:
model.get_topics()

{0: [('know', 0.07573764051107666),
  ('children', 0.06901112028813906),
  ('time', 0.050255556749243896),
  ('love', 0.046079021325495406),
  ('newtown', 0.04171936001037425),
  ('youve', 0.03714901624350276),
  ('thats', 0.03714901624350276),
  ('community', 0.03233112224233856),
  ('comfort', 0.03233112224233856),
  ('say', 0.03233112224233856)],
 1: [('catherine', 0.2910041465176181),
  ('dylan', 0.2910041465176181),
  ('charlotte', 0.2910041465176181),
  ('emilie', 0.2910041465176181),
  ('caroline', 0.2910041465176181),
  ('benjamin', 0.2910041465176181),
  ('avielle', 0.2910041465176181),
  ('ana', 0.2910041465176181),
  ('allison', 0.2910041465176181),
  ('olivia', 0.2910041465176181)]}

In [95]:
model.get_topic(0)

[('europe', 0.11838761852479664),
 ('european', 0.06993116111828732),
 ('countries', 0.05601280761370274),
 ('growth', 0.046183387020634875),
 ('euro', 0.043444506085322784),
 ('problems', 0.03884369644815505),
 ('leaders', 0.035127222356342164),
 ('trading', 0.03406732720752756),
 ('support', 0.03406732720752756),
 ('talking', 0.03406732720752756)]

In [120]:
topics, probs = model.fit_transform(speeches)

In [121]:
model.fit_transform(speeches[97])

TypeError: Make sure that the documents variable is an iterable containing strings only.

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('people', 0.029917703854087773),
 ('thats', 0.02529476998053627),
 ('just', 0.023321096859245927),
 ('america', 0.022370386784902566),
 ('country', 0.020277043703659303),
 ('american', 0.018564529638358567),
 ('make', 0.01824351458931872),
 ('years', 0.017436705863046405),
 ('new', 0.01702567659659818),
 ('weve', 0.016570754872021455)]

In [54]:
topic_model.get_topics()

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{-1: [('people', 0.028771935959897296),
  ('united', 0.02212140544584723),
  ('states', 0.02179716178665238),
  ('shall', 0.019880274323334328),
  ('thats', 0.018785151760315436),
  ('president', 0.01849457471260058),
  ('order', 0.01643421684209892),
  ('america', 0.01642780783078454),
  ('american', 0.015927392795795876),
  ('going', 0.015587511752194316)],
 0: [('people', 0.029917703854087773),
  ('thats', 0.02529476998053627),
  ('just', 0.023321096859245927),
  ('america', 0.022370386784902566),
  ('country', 0.020277043703659303),
  ('american', 0.018564529638358567),
  ('make', 0.01824351458931872),
  ('years', 0.017436705863046405),
  ('new', 0.01702567659659818),
  ('weve', 0.016570754872021455)],
 1: [('thats', 0.025393323470785413),
  ('health', 0.024674986231095437),
  ('care', 0.02313797756895682),
  ('jobs', 0.023040677005758046),
  ('make', 0.02188971444494992),
  ('people', 0.02165835597839326),
  ('new', 0.021063059444611572),
  ('work', 0.020523252670261376),
  ('insu

In [56]:
topic_model.visualize_topics()

/home/muddy/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

/home/muddy/.local/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/home/muddy/.local/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [193]:
lemmatizer = WordNetLemmatizer()
example = ['America', 'American', 'believes', 'thats']
[lemmatizer.lemmatize(ex) for ex in example if ex not in stop_words]

['America', 'American', 'belief']

In [194]:
from nltk.stem import PorterStemmer # similar to lemmatization but without context
stemmer = PorterStemmer()
[stemmer.stem(ex) for ex in first if ex not in stop_words]

['america', 'american', 'belief']

NameError: name 'ex' is not defined